In [1]:

import os
from datetime import datetime

import pandas as pd
import torch
from torch.utils.data import DataLoader

from datasets.SlideSeperatedCSVDataset import SlideSeperatedCSVDataset
from extractors.TemplateMatchExtractor import TemplateMatchExtractor, generate_dataset_from_slides
from labelers.GroundTruthLabeler import GroundTruthLabeler
from models.mlp import MLPBinaryClassifier
from models.resnet import Resnet18BinaryClassifier
from train import train_classifier
from utils import extract_features_from_dataset
from utils import plot_model_metrics
from utils import reduce_dataset, split_dataset


In [2]:
slides_root_dir = "data/whole-slides/gut"
labels_root_dir = "data/labels"
candidates_dataset_dir = "output/candidates"
model_output_dir = "output/models"
PretrainedModelClass = Resnet18BinaryClassifier
features_csv_file_name = f"{PretrainedModelClass.get_pretrained_model_name()}_{PretrainedModelClass.pretrained_output_size}_features.csv"
print(f"{PretrainedModelClass.get_pretrained_model_name()}: {PretrainedModelClass.pretrained_output_size} features")

Resnet18: 512 features


In [3]:
ground_truth_labeler = GroundTruthLabeler(f"{labels_root_dir}/slide-annotations/all.json",
                                          f"{labels_root_dir}/patch-classifications.csv")
extractor = TemplateMatchExtractor(ground_truth_labeler)
generate_dataset_from_slides(slides_root_dir, extractor, candidates_dataset_dir)

Found cached candidates dataset output/candidates


In [4]:
extract_features_from_dataset(candidates_dataset_dir,
                              [Resnet18BinaryClassifier])

Device: cuda:0
Found cached output/candidates/Resnet18_512_features.csv


In [5]:
def assign_categories(dataframe):
    q1, median, q3 = dataframe['n_gt_positive_regions'].quantile([0.25, 0.5, 0.75])

    def categorize_quartiles(n_annotations):
        if n_annotations <= q1:
            return "Low"
        elif q1 < n_annotations <= median:
            return "Medium"
        elif median < n_annotations <= q3:
            return "High"
        else:
            return "Very High"

    dataframe['category'] = dataframe['n_gt_positive_regions'].apply(lambda x: categorize_quartiles(x))
    return dataframe


def split_data(dataframe, train_portion=0.7):
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()
    for category in dataframe['category'].unique():
        category_slides = dataframe[dataframe['category'] == category]
        train_samples = category_slides.sample(frac=train_portion)
        test_samples = category_slides.drop(train_samples.index)
        train_set = pd.concat([train_set, train_samples])
        test_set = pd.concat([test_set, test_samples])
    return train_set, test_set


slides_df = ground_truth_labeler.positive_regions_summary
slides_df = assign_categories(slides_df)
train_slides, test_slides = split_data(slides_df)
print("Train Slides")
train_slides

Train Slides


,slide_name,n_gt_positive_regions,category
4,593453,19,Medium
1,593439,31,Medium
7,593454,22,Medium
0,593444,31,Medium
16,593451,1,Low
21,522021,3,Low
2,593446,15,Low
3,593441,4,Low
5,593438,92,High
12,593450,55,High


In [6]:
print("Test Slides")
test_slides

Test Slides


,slide_name,n_gt_positive_regions,category
10,593434,27,Medium
14,593447,25,Medium
9,593448,13,Low
11,593433,3,Low
20,593437,94,High
8,593436,170,Very High
19,522934,182,Very High


In [7]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

batch_size = 4096
# dataset = SlideSeperatedImageDataset(candidates_dataset_dir, set(train_slides["slide_name"]))
dataset = SlideSeperatedCSVDataset(f"{candidates_dataset_dir}/{features_csv_file_name}",
                                   set(train_slides["slide_name"]))
dataset = reduce_dataset(dataset, discard_ratio=0.0)
train_dataset, validation_dataset = split_dataset(dataset, train_ratio=0.9)
# train_dataset = undersample_dataset(train_dataset)

train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
validation_loader = DataLoader(validation_dataset,
                               batch_size=batch_size,
                               shuffle=True, )

# model = Resnet18BinaryClassifier(hidden_layers=1, units_per_layer=2048,
#                       dropout=0.3, focal_alpha=0.9, focal_gamma=2.0)
model = MLPBinaryClassifier(in_features=PretrainedModelClass.pretrained_output_size, hidden_layers=2,
                            units_per_layer=PretrainedModelClass.pretrained_output_size,
                            dropout=0.3, focal_alpha=0.85, focal_gamma=4.0)

print(f"Dataset: {len(train_dataset):,} training, {len(validation_dataset):,} validation")


Device: cuda:0
Dataset: 144,276 training, 16,031 validation


In [8]:
print(model)

MLPBinaryClassifier(
  (model): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=512, out_features=1, bias=True)
    (10): Sigmoid()
  )
)


In [9]:
print(f"Training starts {datetime.now().isoformat()}")

Training starts 2025-02-06T17:51:30.526412


In [10]:

model = model.to(device)
model, model_metrics = train_classifier(model, train_loader, validation_loader, device,
                                        start_learning_rate=0.000050,
                                        min_learning_rate=0.000025,
                                        lr_warmup_steps=20,
                                        max_epochs=30,
                                        checkpoint_every=1,
                                        eval_every=1)


Epoch 1 training: 100%|██████████| 36/36 [00:02<00:00, 17.53it/s]


Train: 1/30: lr: 0.000050000000 loss:0.002970276662381366


Epoch 1 testing: 100%|██████████| 4/4 [00:00<00:00, 17.81it/s]


Test: 1/30: loss:0.001959073852049187, accuracy:0.9921426408723367, precision:0.0, recall:0.0, f1:0.0, mcc:0.0, ece:0.007857359130866826, epoch:1


Epoch 2 training: 100%|██████████| 36/36 [00:01<00:00, 19.13it/s]


...

Test: 29/30: loss:0.0009491039963904768, accuracy:0.984262256994306, precision:0.2563203463203463, recall:0.5181776556776556, f1:0.3369746369794462, mcc:0.3539951268089094, ece:0.015737742884084582, epoch:29


Epoch 30 training: 100%|██████████| 36/36 [00:03<00:00,  9.61it/s]


Train: 30/30: lr: 0.000025000000 loss:0.0008825884675995136


Epoch 30 testing: 100%|██████████| 4/4 [00:00<00:00, 10.69it/s]

Test: 30/30: loss:0.0009608395484974608, accuracy:0.9869731115269336, precision:0.29821974492789427, recall:0.48428571428571426, f1:0.3674032928197981, mcc:0.3729301641563404, ece:0.013026888482272625, epoch:30


In [11]:


print(f"Training ends {datetime.now().isoformat()}")
for metric in ["accuracy", "precision", "recall", "f1", "mcc"]:
    if len(model_metrics[f"test_{metric}"]) > 0:
        print(f"Test {metric}:", model_metrics[f"test_{metric}"][-1])

plot_model_metrics(model_metrics)

Training ends 2025-02-06T17:53:26.513654
Test accuracy: 0.9869731115269336
Test precision: 0.29821974492789427
Test recall: 0.48428571428571426
Test f1: 0.3674032928197981
Test mcc: 0.3729301641563404


In [12]:
os.makedirs(model_output_dir, exist_ok=True)
torch.save(model, f"{model_output_dir}/model.pickle")
torch.save({
    "train_slides": set(train_slides["slide_name"]),
    "test_slides": set(test_slides["slide_name"])
},
    f"{model_output_dir}/data-split.pickle"
)